In [19]:
import json


# Load the JSON file (adjust path if needed)
with open('/content/drive/MyDrive/Colab Notebooks/PROJECT 2/intents.json') as file:
    data = json.load(file)

# Preview the data
import pprint
pprint.pprint(data)


{'intents': [{'patterns': ['Hi',
                           'Hello',
                           'Hey',
                           'Good morning',
                           'Good evening'],
              'responses': ['Hello!', 'Hi there!', 'Hey!', 'Greetings!'],
              'tag': 'greeting'},
             {'patterns': ['Bye',
                           'Goodbye',
                           'See you later',
                           'Catch you later'],
              'responses': ['Bye!', 'See you soon!', 'Take care!', 'Goodbye!'],
              'tag': 'goodbye'},
             {'patterns': ['Thanks', 'Thank you', 'Appreciate it'],
              'responses': ["You're welcome!", 'Glad to help!', 'Anytime!'],
              'tag': 'thanks'},
             {'patterns': ['What are your hours?',
                           'When are you open?',
                           'Opening hours?'],
              'responses': ["We're open from 9am to 5pm, Monday to Friday."],
              'tag': 'hou

In [20]:
import shutil
shutil.rmtree('/root/nltk_data', ignore_errors=True)


In [21]:
import numpy as np
import random
import nltk
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')

stemmer = PorterStemmer()

import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download the punkt_tab dataset

all_words = []
tags = []
xy = []

for intent in data['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [23]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

ignore_words = ['?', '!', '.', ',']
all_words = [stemmer.stem(w.lower()) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))


In [24]:
import numpy as np

X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = []
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_sentence]
    for w in all_words:
        bag.append(1 if w in pattern_words else 0)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [25]:
!pip install tensorflow


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [27]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(tags), activation='softmax'))


In [28]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.01),
              metrics=['accuracy'])


In [29]:
hist = model.fit(X_train, y_train, epochs=200, batch_size=8, verbose=1)


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.1892 - loss: 1.6490
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 1.4221
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6024 - loss: 1.1894
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8698 - loss: 1.0893
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8385 - loss: 0.8322
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8507 - loss: 0.6722 
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9722 - loss: 0.4623 
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8819 - loss: 0.3333 
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 0.1988 
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9253 - loss: 0.1834 
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 1.0000 - loss: 0.0582
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 1.0000

In [30]:
model.save("chatbot_model.keras")


In [31]:
import pickle

data = {
    "all_words": all_words,
    "tags": tags,
    "X_train": X_train,
    "y_train": y_train
}

with open("chatbot_data.pkl", "wb") as f:
    pickle.dump(data, f)


In [32]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, all_words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(all_words)
    for w in sentence_words:
        for i, word in enumerate(all_words):
            if word == w:
                bag[i] = 1
    return np.array(bag)


In [34]:
import json

with open("/content/drive/MyDrive/Colab Notebooks/PROJECT 2/intents.json") as file:
    intents = json.load(file)


In [35]:
def chat():
    print("Start talking with the bot! (type 'quit' to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        bow = bag_of_words(inp, all_words)
        bow = np.expand_dims(bow, axis=0)
        result = model.predict(bow)[0]
        idx = np.argmax(result)
        tag = tags[idx]

        if result[idx] > 0.7:
            for intent in intents['intents']:
                if intent['tag'] == tag:
                    responses = intent['responses']
            print("Bot:", random.choice(responses))
        else:
            print("Bot: I didn't understand that. Try again.")


In [36]:
chat()


Start talking with the bot! (type 'quit' to stop)
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Bot: Hey!
You: how are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Bot: We're open from 9am to 5pm, Monday to Friday.
You: thank you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bot: Anytime!
You: what are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bot: We're open from 9am to 5pm, Monday to Friday.
You: quit
